# Tutorial

1. Point to the root directory containing all .txt files 
2. Output is saved to ```output``` dir.

In [23]:
!python darknet_label_remapper.py -i /home/david/Documents/Datasets/STS_COCO_dataset_v1/labels -l old_label.label -nl new_label.label -r sample_remap.label

{'traffic_sign': 0, 'pedestrian': 1, 'bicycle': 2, 'car': 3, 'motocycle': 4, 'other_vehicle': 5, 'bus': 6, 'truck': 7, 'traffic_light': 8, 'animal': 9}
Namespace(input='/home/david/Documents/Datasets/STS_COCO_dataset_v1/labels', label='old_label.label', new_label='new_label.label', remapper='sample_remap.label', walk=None)
 97%|████████████████████████████████ | 135411/139327 [00:21<00:00, 5764.10it/s]output/1277107089Image000056.txt
['0 0.289211313671875 0.4261012942708334 0.016538616406249984 0.02228360937500001', '0 0.2888120765625 0.44544923229166666 0.015479651562499974 0.017964039583333324', '0 0.015295816796875 0.4461123973958333 0.01530532421875 0.019606819791666653', '0 0.016172130859375 0.4666139375 0.013326839843749999 0.016954022916666686']
100%|█████████████████████████████████| 139327/139327 [00:22<00:00, 6314.66it/s]
Saved to output
Finished.


3. To use darknet2kitti.py, we want the .txt files and .jpg files to conform to darknet structure. We move all relevant images to output folder

In [24]:
import os, glob, shutil
from pathlib import Path
import tqdm

In [25]:
INPUT_DIR = "output" # Containing the .txt files

TRAIN_IMAGE_OUTPUT_DIR = "train/images"
TRAIN_LABEL_OUTPUT_DIR = "train/labels"

VALID_IMAGE_OUTPUT_DIR = "valid/images"
VALID_LABEL_OUTPUT_DIR = "valid/labels"


Path(TRAIN_IMAGE_OUTPUT_DIR).mkdir(exist_ok=True, parents=True)
Path(TRAIN_LABEL_OUTPUT_DIR).mkdir(exist_ok=True, parents=True)

Path(VALID_IMAGE_OUTPUT_DIR).mkdir(exist_ok=True, parents=True)
Path(VALID_LABEL_OUTPUT_DIR).mkdir(exist_ok=True, parents=True)

# STS specific operation

We sort images and slice the validaiton set 

In [26]:
# Save path to original images into an array
imgs = glob.glob("/home/david/Documents/Datasets/STS_COCO_dataset_v1/images/coco_train2017/*.jpg")
imgs += glob.glob("/home/david/Documents/Datasets/STS_COCO_dataset_v1/images/unlabelled_sts/*.jpg")
imgs += glob.glob("/home/david/Documents/Datasets/STS_COCO_dataset_v1/images/batch0/*.jpg")

In [27]:
imgs.sort(reverse=True,key=lambda x: os.path.basename(x)) 
valid_images = imgs[:3000]
train_images = imgs[3000:]

# COCO samples inside validation set
for path in valid_images:
    assert "batch0" in path or "unlabelled_sts" in path

In [28]:
for img_path in tqdm.tqdm(train_images):
    img_filename = os.path.basename(img_path)
    label_filename = img_filename.replace(".jpg", ".txt")
    
    label_path = os.path.join(INPUT_DIR, label_filename)
    assert os.path.isfile(label_path)
    
    new_img_path = os.path.join(TRAIN_IMAGE_OUTPUT_DIR, img_filename)
    new_label_path = os.path.join(TRAIN_LABEL_OUTPUT_DIR, label_filename)
    
    shutil.copyfile(img_path, new_img_path)
    shutil.copyfile(label_path, new_label_path)
    
for img_path in tqdm.tqdm(valid_images):
    img_filename = os.path.basename(img_path)
    label_filename = img_filename.replace(".jpg", ".txt")
    
    label_path = os.path.join(INPUT_DIR, label_filename)
    assert os.path.isfile(label_path)
    
    new_img_path = os.path.join(VALID_IMAGE_OUTPUT_DIR, img_filename)
    new_label_path = os.path.join(VALID_LABEL_OUTPUT_DIR, label_filename)
    
    shutil.copyfile(img_path, new_img_path)
    shutil.copyfile(label_path, new_label_path)

100%|██████████| 3000/3000 [00:05<00:00, 576.53it/s]


4. Now we use darknet2kitti.py in order to create the kitti dataset

In [29]:
!python darknet2kitti.py -i train/labels/ -l new_label.label -tlt
!python darknet2kitti.py -i valid/labels/ -l new_label.label -tlt

100%|█████████████████████████████████| 134523/134523 [00:32<00:00, 4159.87it/s]
Output saved to output/train/labels/000000294667.txt
Finished.
100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 9200.46it/s]
Output saved to output/valid/labels/1277387763Image000023.txt
Finished.


5. Clean

In [30]:
!cp -r output/train .
!cp -r output/valid .

In [31]:
for path in glob.glob("output/*.txt"):
    os.remove(path)

In [32]:
!rm -r output

In [116]:
#!rm -r train
#!rm -r valid